# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given.
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [2]:
import re
import unittest
import random

from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```).


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [3]:
def dna_to_rna(s):
    """
    Transcribes a DNA sequence into the corresponding RNA sequence by replacing 'T' with 'U'.
    """
    return s.replace("T", "U")


In [4]:
class TestDnaToRna(unittest.TestCase):
    def test_mixed_sequence(self):
        self.assertEqual(dna_to_rna("AACGTGATTTC"), "AACGUGAUUUC")

    def test_only_thymines(self):
        self.assertEqual(dna_to_rna("TTTT"), "UUUU")

    def test_all_bases(self):
        self.assertEqual(dna_to_rna("ACGT"), "ACGU")

    def test_no_thymine(self):
        self.assertEqual(dna_to_rna("GGGCCC"), "GGGCCC")

    def test_empty_string(self):
        self.assertEqual(dna_to_rna(""), "")


if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)

test_all_bases (__main__.TestDnaToRna.test_all_bases) ... ok
test_empty_string (__main__.TestDnaToRna.test_empty_string) ... ok
test_mixed_sequence (__main__.TestDnaToRna.test_mixed_sequence) ... ok
test_no_thymine (__main__.TestDnaToRna.test_no_thymine) ... ok
test_only_thymines (__main__.TestDnaToRna.test_only_thymines) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.012s

OK


### Idea of solution

The function implements transcription by directly replacing every occurrence of ```T``` with ```U``` in the DNA string using Python’s built-in ```str.replace``` method. This is efficient and concise, since all other bases remain unchanged.

### Discussion

The program was tested with several cases: mixed sequences, only thymine, all four bases, sequences without thymine, and an empty string. In all cases the results were correct, e.g. ```AACGTGATTTC``` was transcribed to ```AACGUGAUUUC```. The solution is simple, runs in linear time relative to the length of the input, and works reliably for valid uppercase DNA input.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time.

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [5]:
conversion_table = """UUU F 0.46 17.6 (714298)  UCU S 0.19 15.2 (618711)  UAU Y 0.44 12.2 (495699)  UGU C 0.46 10.6 (430311)
                      UUC F 0.54 20.3 (824692)  UCC S 0.22 17.7 (718892)  UAC Y 0.56 15.3 (622407)  UGC C 0.54 12.6 (513028)
                      UUA L 0.08  7.7 (311881)  UCA S 0.15 12.2 (496448)  UAA * 0.30  1.0 ( 40285)  UGA * 0.47  1.6 ( 63237)
                      UUG L 0.13 12.9 (525688)  UCG S 0.05  4.4 (179419)  UAG * 0.24  0.8 ( 32109)  UGG W 1.00 13.2 (535595)

                      CUU L 0.13 13.2 (536515)  CCU P 0.29 17.5 (713233)  CAU H 0.42 10.9 (441711)  CGU R 0.08  4.5 (184609)
                      CUC L 0.20 19.6 (796638)  CCC P 0.32 19.8 (804620)  CAC H 0.58 15.1 (613713)  CGC R 0.18 10.4 (423516)
                      CUA L 0.07  7.2 (290751)  CCA P 0.28 16.9 (688038)  CAA Q 0.27 12.3 (501911)  CGA R 0.11  6.2 (250760)
                      CUG L 0.40 39.6 (1611801)  CCG P 0.11  6.9 (281570)  CAG Q 0.73 34.2 (1391973)  CGG R 0.20 11.4 (464485)

                      AUU I 0.36 16.0 (650473)  ACU T 0.25 13.1 (533609)  AAU N 0.47 17.0 (689701)  AGU S 0.15 12.1 (493429)
                      AUC I 0.47 20.8 (846466)  ACC T 0.36 18.9 (768147)  AAC N 0.53 19.1 (776603)  AGC S 0.24 19.5 (791383)
                      AUA I 0.17  7.5 (304565)  ACA T 0.28 15.1 (614523)  AAA K 0.43 24.4 (993621)  AGA R 0.21 12.2 (494682)
                      AUG M 1.00 22.0 (896005)  ACG T 0.11  6.1 (246105)  AAG K 0.57 31.9 (1295568)  AGG R 0.21 12.0 (486463)

                      GUU V 0.18 11.0 (448607)  GCU A 0.27 18.4 (750096)  GAU D 0.46 21.8 (885429)  GGU G 0.16 10.8 (437126)
                      GUC V 0.24 14.5 (588138)  GCC A 0.40 27.7 (1127679)  GAC D 0.54 25.1 (1020595)  GGC G 0.34 22.2 (903565)
                      GUA V 0.12  7.1 (287712)  GCA A 0.23 15.8 (643471)  GAA E 0.42 29.0 (1177632)  GGA G 0.25 16.5 (669873)
                      GUG V 0.46 28.1 (1143534)  GCG A 0.11  7.4 (299495)  GAG E 0.58 39.6 (1609975)  GGG G 0.25 16.5 (669768)"""

In [6]:
def get_dict():
    """
    Parses the codon usage table and returns a dictionary mapping each RNA codon (triplet)
    to its corresponding amino acid (one-letter code).
    """
    pattern = r"([AUCG]{3})\s+([A-Z*])"
    matches = re.findall(pattern, conversion_table)
    return {triplet: aa for triplet, aa in matches}


if __name__ == "__main__":
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

We construct a dictionary mapping “codon → amino acid” by parsing the text table with a regular expression. The pattern `([AUCG]{3})\s+([A-Z*])` captures an RNA triplet (A/U/C/G) followed by the one-letter amino acid code (including stop codons `*`). From the extracted pairs `(codon, amino acid)` we build a `dict`. This approach is simple, requires no external libraries, and runs in linear time relative to the length of the input string.



### Discussion

- The solution ignores frequencies and counts in parentheses, keeping only the essential mapping between codons and one-letter amino acid codes.  
- All 64 codons are supported, including the stop codons `UAA`, `UAG`, and `UGA` (mapped to `'*'`).  
- Case sensitivity: the input is expected to be uppercase RNA bases (A/U/C/G).  
- Complexity: O(N), where N is the length of the table string; memory usage is O(1) with respect to the fixed number of possible keys (maximum 64).  


3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [7]:
def get_dict_list():
    """
    Parses the codon usage table and returns a dictionary mapping each amino acid
    (one-letter code) to the list of RNA codons (triplets) that encode it.
    """
    pattern = r"([AUCG]{3})\s+([A-Z*])"
    matches = re.findall(pattern, conversion_table)
    amino_acid_dict = defaultdict(list)
    for triplet, aa in matches:
        amino_acid_dict[aa].append(triplet)
    return dict(amino_acid_dict)


if __name__ == "__main__":
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

We invert the mapping from the previous task. Using the same regular expression pattern `([AUCG]{3})\s+([A-Z*])` to extract `(codon, amino_acid)` pairs, we build a dictionary where **each amino acid** is the key and the **value is the list of all codons** that encode it. Practically, we iterate over all matches and append each codon to `aa_to_codons[amino_acid]`. A `defaultdict(list)` makes this concise and efficient. The approach is linear in the size of the input table and requires no external dependencies.


### Discussion

- The output dictionary has one entry per unique amino acid (20 standard amino acids plus the stop symbol `*`).
- Each value lists all synonymous codons for that amino acid (e.g., `L` maps to six codons; `*` maps to `UAA`, `UAG`, `UGA`).
- Complexity is O(N) over the length of the table string; memory is bounded by the genetic code (64 codons).


With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence.
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [8]:
def rna_to_prot(s):
    """
    Translates an RNA sequence into the corresponding protein sequence
    using the codon-to-amino-acid mapping. Translation stops at the first stop codon.
    """
    mapping = get_dict()
    protein = []

    for i in range(0, len(s) - 2, 3):
        codon = s[i : i + 3]
        aa = mapping.get(codon, "")
        if aa == "*":
            break
        protein.append(aa)

    return "".join(protein)


def dna_to_prot(s):
    """
    Translates a DNA sequence into the corresponding protein sequence
    (via RNA intermediate).
    """
    return rna_to_prot(dna_to_rna(s))


if __name__ == "__main__":
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM


### Idea of solution

We implement protein translation by first converting DNA into RNA (replacing `T` with `U`) and then reading the RNA sequence in codons (triplets of nucleotides). Each codon is mapped to its corresponding amino acid using the dictionary from the previous exercises. The translation proceeds codon by codon, appending amino acids until a stop codon (`UAA`, `UAG`, `UGA`) is encountered, at which point translation terminates. This ensures the produced protein matches the biological process where ribosomes stop elongation at stop signals.


### Discussion

- The approach cleanly separates two steps: DNA → RNA transcription and RNA → protein translation.
- The solution leverages the codon-to-amino acid mapping dictionary, which makes the translation process concise and efficient (linear time with respect to sequence length).
- Stop codons are correctly handled: translation halts and does not include them in the output.
- Testing with the sequence `ATGATATCATCGACGATGTAG` yields `MISSTM`, demonstrating that the method works as expected.


You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [9]:
def get_codon_counts():
    """
    Extracts raw observation counts for each RNA codon from the conversion table.
    Returns: dict {codon -> count:int}
    """
    pattern = r"([AUCG]{3})\s+[A-Z*].*?\((\d+)\)"
    matches = re.findall(pattern, conversion_table)

    codon_counts = {}
    for codon, count in matches:
        codon_counts[codon] = int(count)
    return codon_counts


def get_probabability_dict():
    """
    Builds a probability dictionary for each codon given its amino acid group.
    For each amino acid, probabilities of its synonymous codons sum to 1.
    Returns: dict {codon -> probability:float}
    """
    codon_counts = get_codon_counts()
    aa_to_codons = get_dict_list()

    probabilities = {}

    for codons in aa_to_codons.values():
        total = sum(codon_counts.get(c, 0) for c in codons)
        if total == 0:
            p = 1.0 / len(codons)
            for c in codons:
                probabilities[c] = p
        else:
            for c in codons:
                probabilities[c] = codon_counts.get(c, 0) / total

    return probabilities


if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.000000	UAC: 0.556662	UAG: 1.000000	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.000000	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The task is to compute codon usage probabilities from observed counts.  
We first parse the codon usage table and extract, for each codon, the absolute count of times it was observed (the number inside parentheses). Then, for each amino acid, we collect all its synonymous codons and compute probabilities by normalizing counts within that group. This produces a dictionary where each codon (e.g., `"AUC"`) maps to its probability among all codons for the same amino acid. The solution uses `defaultdict` and iteration over amino acid groups to ensure the probabilities of synonymous codons sum to 1.


### Discussion

- The approach respects **codon bias**: some codons are more likely to appear than others even though they encode the same amino acid.  
- Using absolute counts from the parentheses column avoids rounding errors present in the truncated frequency columns.  
- The resulting dictionary covers all 64 codons, including start and stop codons. For codons with zero counts (like some stop codons), the probability is set to `0.0`.  
- This representation allows sampling codons randomly in proportion to their biological frequencies, which is useful for tasks such as simulating mRNA sequences or studying codon adaptation.  
- Example: for isoleucine, codons `AUU`, `AUC`, `AUA` have probabilities 0.361, 0.470, and 0.169 respectively, reflecting their observed distribution.  
- Complexity is linear in the number of codons (64), so the method is efficient. Improvements could include error handling for missing codons and normalizing input automatically to uppercase.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToMaxRNA:
    """
    Builds a mapping from amino acids to their most probable codons.
    """

    def __init__(self):
        """
        Initialize the dictionary of best codons using probability data.
        """
        aa_to_codons = get_dict_list()
        codon_prob = get_probabability_dict()
        self.best_codon = {aa: max(codons, key=lambda c: codon_prob.get(c, 0.0)) for aa, codons in aa_to_codons.items()}

    def convert(self, s):
        """
        Convert a protein sequence (amino acids) into an RNA sequence.
        """
        return "".join(self.best_codon[aa] for aa in s)

if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

We already have two helper functions:  
- `get_dict_list()` builds a dictionary {amino_acid → list of codons}.  
- `get_probabability_dict()` calculates probabilities for each codon so that all codons of one amino acid sum to 1.  

The algorithm is straightforward: for each amino acid in the protein sequence, we choose the codon with the highest probability among its synonyms. This selection is precomputed in the constructor of the class (`self.best_codon`). Then the `convert` method simply replaces each amino acid with its most probable codon and concatenates the results into a single RNA string.


### Discussion

This solution essentially inverts the usual translation process: instead of converting RNA to protein, we reconstruct the **most likely RNA sequence** for a given protein.  

The method runs in linear time with respect to the length of the protein sequence, since each amino acid requires just one dictionary lookup. The codon chosen depends on the codon usage frequencies from the input table (`conversion_table`), which makes the reconstruction biologically more realistic (different organisms prefer different codons, known as **codon bias**).  

A limitation is that the method always returns only one “most probable” RNA sequence. In reality, many synonymous RNA sequences can encode the same protein, but here we deterministically pick the single most likely option.


Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively.
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [11]:
import random
import numpy as np

def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    events = list(dist.keys())
    probs = list(dist.values())

    cumsum = np.cumsum(probs)

    r = random.random()

    for i, bound in enumerate(cumsum):
        if r < bound:
            return events[i]

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

T, G, G, T, T, T, C, C, C, T, G, T, T, C, T, C, T, T, G, C, C, C, A, T, T, T, C, A, C


### Idea of solution

The core idea is to transform the probability distribution into cumulative probabilities, thereby partitioning the interval [0,1) into segments whose lengths correspond to the event probabilities. We then generate a random number in [0,1) and select the event associated with the first cumulative bound greater than the random number. This ensures each event is chosen proportionally to its probability.

### Discussion

This method is simple, efficient, and widely used for sampling from discrete probability distributions. The cumulative approach guarantees that the sum of probabilities covers the whole [0,1) interval and that each event is represented correctly. One must be careful about floating-point inaccuracies: if the probabilities do not sum exactly to 1, scaling or a small tolerance may be required. The time complexity is O(n) in the number of events, which is sufficient for small distributions. For very large distributions, more advanced methods such as binary search on cumulative sums or alias sampling can be used to speed up the selection.

With this general routine, the following should be easy to solve.

8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [33]:
class ProteinToRandomRNA(object):
    """
    Produces a random RNA sequence encoding the input protein.
    Each amino acid is mapped to a codon sampled by adaptation probabilities.
    """

    def __init__(self):
        self.aa_to_codons = get_dict_list()
        self.codon_prob = get_probabability_dict()

    def get_dist(self, codons):
        probs = [self.codon_prob[codon] for codon in codons]
        return dict(zip(codons, probs))


    def convert(self, s):
        """
        Sample a codon for each amino acid in `protein` and return the RNA string.
        """
        self.random_codon = {aa: random_event(self.get_dist(codons)) for aa, codons in self.aa_to_codons.items()}
        return "".join(self.random_codon[aa] for aa in s)

if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUCACACCCAUACAAAACCGGGCU


### Idea of solution

Transform codon adaptation probabilities into per–amino-acid sampling distributions and use a simple cumulative-probability sampler (random_event). For each amino acid in the input protein, select a codon at random in proportion to its probability and concatenate the sampled codons to form the RNA sequence. The randomness ensures that repeated runs can yield different valid encodings while still respecting the given probabilities.

### Discussion

This approach samples from a discrete distribution via cumulative probabilities, giving each codon a selection chance equal to its adaptation probability. Time complexity is linear in the protein length
𝐿
L (plus a small per-AA overhead). To improve robustness, probabilities may be normalized per amino acid and unknown amino acids should be handled explicitly (e.g., error or uniform fallback). If reproducibility is needed, seeding the RNG is advisable. Note that pre-sampling a single codon per amino acid (and reusing it) reduces variability within a run; sampling per position preserves the intended stochasticity across all occurrences.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences.
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution.
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one.

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [13]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    for _ in s:
        yield {}


if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


### Idea of solution

fill in

### Discussion

fill in


Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*.
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position.

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [14]:
def context_list(s, k):
    return {}

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{}


### Idea of solution

fill in

### Discussion

fill in

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [15]:
def context_probabilities(s, k):
    return {}

if __name__ == '__main__':
    pass

### Idea of solution

fill in

### Discussion

fill in

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities.

In [16]:
class MarkovChain:

    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def generate(self, n, seed=None):
        return "$" * n

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

$$$$$$$$$$


### Idea of solution

fill in

### Discussion

fill in

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [17]:
def context_pseudo_probabilities(s, k):
    return {"": ""}

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))

    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: 
: {}

 $$$$$$$$$$$$$$$$$$$$


### Idea of solution

fill in

### Discussion

fill in

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [18]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def probability(self, s):
        return np.nan


if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is nan


### Idea of solution

fill in

### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform.
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [19]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        pass

    def log_probability(self, s):
        return np.nan

if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is nan


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies.

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [20]:
def better_context_probabilities(s, k):
    return {"": ""}

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

: 


### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution:
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability.

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [21]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        pass

    def generate(self, n, seed=None):
        return "Q"*n

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

QQQQQQQQQQ


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [22]:
def kmer_index(s, k):
    return {}

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{}


### Idea of solution

fill in

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended.

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$.

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [23]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): 0 for codon in product("ACGU", repeat=3)}

def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return np.nan

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000

    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))
    pass

    # Maybe check that converting back to protein results in the same sequence
    pass

    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities("<rna sequence>") # placeholder call

    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict

    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call

    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = nan
d(predicted || original) = nan

d(original || uniform) = nan
d(uniform || original) = nan

d(predicted || uniform) = nan
d(uniform || predicted) = nan


### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [24]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5


if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p)
        for p in get_stationary_distributions(transition)))

-0.172, +0.405, -0.467, -0.103
+0.248, +0.475, -0.061, -0.312


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [25]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[-0.23055953  0.10382718 -0.27458709 -0.14068193]
 [ 0.38515224  0.48844312  0.04581161  0.17699008]]
Using [0.39, 0.49, 0.05, 0.18] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.59413000
KL divergence of stationary distribution prefix of length    10 is 0.09258067
KL divergence of stationary distribution prefix of length   100 is 0.69268376
KL divergence of stationary distribution prefix of length  1000 is 0.41582823
KL divergence of stationary distribution prefix of length 10000 is 0.69704461


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [26]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5,
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[ 0.03104708  0.42420569 -0.48321317  0.24507583]

Using [-0.08794098  0.46284033 -0.38554649 -0.36384751] as initial distribution:
kl-divergence   empirical distribution
0.78003113465   [0.36501267 0.12009476 0.30242836 0.13718875]
0.67214154599   [0.23867641 0.27547495 0.14453306 0.08587349]
0.79772127302   [ 0.11956405 -0.20510922 -0.34984506  0.37833414]
0.23013413633   [-0.0257633  -0.03088234  0.35935892  0.08199747]
0.38640849172   [-0.11816349 -0.45837739 -0.20102422 -0.11464482]

Using [-0.39050732 -0.43517827  0.26740709 -0.34365933] as initial distribution:
kl-divergence   empirical distribution
0.52146021973   [ 0.24278404 -0.04551347  0.27932431  0.20619453]
0.11066968709   [ 0.25609664  0.31348852 -0.08257537 -0.27335366]
0.86320166898   [-0.4341002  -0.37299151  0.45177169  0.44015131]
0.37014772164   [ 0.17422885 -0.3822963

### Idea of solution

fill in

### Discussion
fill in